In [1]:
import re
import os
import pandas as pd
import random

/home/prevot/anaconda3/lib/python3.11/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [2]:
FILE_DISFLUENT_BASE = './data_benchmark/real_disf.txt'
#FILE_FLUENT_BASE = './data_benchmark/real_fluent.txt'

In [3]:
DISFLUENT_EXAMPLES = [u[:-1] for u in open(FILE_DISFLUENT_BASE,'r').readlines()]
#FLUENT_EXAMPLES = [u[:-1] for u in open(FILE_FLUENT_BASE,'r').readlines()]

#DISFLUENT_DF_LENGTHS = pd.Series([len(u.split()) for u in DISFLUENT_EXAMPLES]).plot(kind='box')

In [5]:
DISFLUENT_DF = pd.DataFrame(DISFLUENT_EXAMPLES,columns=['text'])
#DISFLUENT_DF['disfluent'] =True

#FLUENT_DF = pd.DataFrame(FLUENT_EXAMPLES,columns=['text'])
#FLUENT_DF['disfluent'] =False


In [41]:
#ALL_DF = pd.concat([DISFLUENT_DF,FLUENT_DF],axis=0) 
#ALL_DF.to_csv('disfluency_fr.csv',index=False)

In [29]:
DISF_ITEMS = ['euh']

# MODIFYING GENUINE DISFLUENT EXAMPLES
def move_disfluencies(row):
    utt_as_list = row['text'].split()
    filtered_utt = [t for t in utt_as_list if t not in DISF_ITEMS]
    items_to_move = [t for t in utt_as_list if t in DISF_ITEMS]
    new_utt = utt_as_list
    while new_utt == utt_as_list :
        new_utt = filtered_utt
        for item in items_to_move:
            insert_location = random.randrange(2, len(new_utt)-2)
            new_utt.insert(insert_location,item) 
    return ' '.join(new_utt)

def remove_disfluencies(row):
    utt_as_list = row['text'].split()
    return ' '.join([t for t in utt_as_list if t not in DISF_ITEMS])


def shuffle_utterance(row):
    utt_as_list = row['text'].split()
    random.shuffle(utt_as_list)
    return ' '.join(utt_as_list)


In [60]:
######
# DEPRECATED
######

# MODIFYING FLUENT EXAMPLES
def inject_fake_disf(row,filler='euh'):
    utt_as_list = row['text'].split()
    number = int(max(1,int(len(utt_as_list) / 5))) 
    for x in range(0, number): # this line will ensure 4 element insertion
        insert_location = random.randrange(2, len(utt_as_list)-2)
        utt_as_list.insert(insert_location,filler)
        if random.random()> 0.5: 
            utt_as_list.insert(insert_location+1,'#')
    return ' '.join(utt_as_list)

In [30]:
DISFLUENT_DF['moved_disf'] = DISFLUENT_DF.apply(move_disfluencies,axis=1)
DISFLUENT_DF['removed_disf'] = DISFLUENT_DF.apply(remove_disfluencies,axis=1)
DISFLUENT_DF['shuffled'] = DISFLUENT_DF.apply(shuffle_utterance,axis=1)

In [31]:
DISFLUENT_DF

,text,moved_disf,removed_disf,shuffled
0,c' était un bar avec euh un un écran ou #,c' était euh un bar avec un un écran ou #,c' était un bar avec un un écran ou #,un euh écran ou c' était un un # avec bar
1,je crois j' étais euh je me rappelle plus si j...,je crois j' étais je me rappelle plus si j' ét...,je crois j' étais je me rappelle plus si j' ét...,si étais étais fac crois en # # rappelle on me...
2,on a appelé euh des # * les parents d' amis #,on a appelé des # euh * les parents d' amis #,on a appelé des # * les parents d' amis #,on amis d' a les des # euh # appelé * parents
3,mais jetés comme des euh # comme des miséreux ...,mais jetés comme des # comme des euh miséreux ...,mais jetés comme des # comme des miséreux sur ...,euh comme comme mais # # la des sur bouffe jet...
4,on en pouvait plus de euh d' attendre *,on en pouvait plus euh de d' attendre *,on en pouvait plus de d' attendre *,d' de * on en plus euh pouvait attendre
...,...,...,...,...
195,le fait d' avoir la ville et puis le fait d' a...,le fait d' avoir la ville et puis le fait d' a...,le fait d' avoir la ville et puis le fait d' a...,ville des de de la nature le fait puis de à et...
196,ah pff ah moi j' ai pas envie de repartir euh ...,ah pff ah moi j' ai pas envie de repartir # de...,ah pff ah moi j' ai pas envie de repartir # de...,euh ah envie de la de moi de # pff de de # j' ...
197,et qu' ils aient pas euh # euh # un terrain eu...,et qu' euh ils aient pas # euh euh # un terrai...,et qu' ils aient pas # # un terrain dans leque...,ils vois euh terrain pas un et # euh euh jouer...
198,mais euh # moi j' avais un j' avais un voisin ...,mais # moi j' avais un j' avais un voisin qui ...,mais # moi j' avais un j' avais un voisin qui ...,habitait euh mais de avais côté voisin un j' q...


In [51]:
result = []
cnt=0
for i,row in DISFLUENT_DF.iterrows():
    item_moved = {"sentence_good":row['text'], "sentence_bad":row['moved_disf'], "field": "disfluencies", 
                 "linguistics_term": "moved_filler", "UID": "moved_filler", "simple_LM_method": True,
                 "one_prefix_method": False, "two_prefix_method": False, 
                 "lexically_identical": True, "pair_id": str(cnt)}
    result.append(item_moved)
    cnt +=1
    item_removed = {"sentence_good":row['text'],"sentence_bad":row['removed_disf'], "field": "disfluencies", 
                    "linguistics_term": "removed_filler", "UID": "removed_filler", "simple_LM_method": True,
                    "one_prefix_method": False, "two_prefix_method": False, 
                    "lexically_identical": False, "pair_id": str(cnt)}
    result.append(item_removed)
    cnt +=1
    item_shuffled = {"sentence_good":row['text'],"sentence_bad":row['shuffled'], "field": "disfluencies", 
                     "linguistics_term": "shuffled", "UID": "shuffled", "simple_LM_method": True, 
                     "one_prefix_method": False, "two_prefix_method": False, "lexically_identical": True, 
                     "pair_id": str(cnt)}
    result.append(item_shuffled)
    cnt +=1

In [52]:
with open('data_benchmark/disfluency_fr.json', 'w', encoding='utf-8') as outfile:
    for entry in result:
        json.dump(entry, outfile, ensure_ascii=False)
        outfile.write('\n')